In [ ]:
class Sentiment:
    POSITIVE = 'POSITIVE'
    NEGATIVE = 'NEGATIVE'
    NEUTRAL = 'NEUTRAL'



class Review:
    def __init__(self,text,rating):
        self.text=text
        self.rating=rating
        self.sentiment=self.get_sentiment()


    def get_sentiment(self):
        if self.rating <=2:
            return Sentiment.NEGATIVE
        elif self.rating == 3:
            return Sentiment.NEUTRAL
        else:
            return Sentiment.POSITIVE

In [89]:
import json
file_name = 'Books_small.json'

# Update the get_sentiment method in the Review class

reviews=[]
with open(file_name) as f:
    for line in f:
       review = json.loads(line)
       reviews.append(Review(review['reviewText'], review['overall']))
    
reviews[10].sentiment


'positive'

prep Data

In [96]:
from sklearn.model_selection import train_test_split
training,test = train_test_split(reviews,test_size=0.33,random_state=42)  # 33% of the reviews will be for testing



In [105]:
# Extracting the text of each review from the training dataset
train_x = [x.text for x in training]

# Extracting the sentiment of each review from the training dataset
train_y = [x.sentiment for x in training]

train_x[7]
train_y[7]



'negative'